In [ ]:
# Template Notebook for Engagement Classification Model

# ToDo/Models to run here: https://docs.google.com/document/d/18EJpFesMEl9Q7C1AZzDeq6uy7c8tDMvEJ_j58-MmiBI/edit?usp=sharing
# Use p8_data_processed.csv here: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [1]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [54]:
# Load Data
# Warning: this will probably take some time
# Adjust file path based on your computer

file8 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p8_data_FS4.csv'
file3 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p3_data_FS4.csv'
file7 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p7_data_FS4.csv'
file5 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p5_data_FS4.csv'
file9 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p9_data_FS4.csv'
file10 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p10_data_FS4.csv'

data = pd.read_csv(file3)


In [45]:
data.head()

,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,...,ts_attempt,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,0.94898,1.0,0.707127,0.517393,0.045885,0.536258,0.576956,0.006457,0.507990,0.517745,...,0.0,0,0,0,0,0,0,0,0,1
1,0.94898,1.0,0.705361,0.517181,0.045066,0.559994,0.587393,0.010422,0.512330,0.519512,...,0.0,0,0,0,0,0,0,0,0,1
2,0.94898,1.0,0.702433,0.522912,0.044037,0.567031,0.591571,0.012055,0.513119,0.521118,...,0.0,0,0,0,0,0,0,0,0,1
3,0.94898,1.0,0.696354,0.520243,0.041216,0.569312,0.616382,0.017313,0.512527,0.524811,...,0.0,0,0,0,0,0,0,0,0,1
4,0.94898,1.0,0.696378,0.516838,0.041063,0.564469,0.609426,0.015121,0.511541,0.523045,...,0.0,0,0,0,0,0,0,0,0,1


Note: keep timestamp and session_num until right before running the model in all feature sets

### Step 1: Train Test Split 

Train on Earlier Sessions, Test on Later Sessions 70-30

In [55]:
# Feature Set 4 is used
FS = data

In [56]:
# sort values for this sequential model
FS = FS.sort_values(['session_num', 'timestamp'])

In [57]:
for val in FS['session_num']:
    if val not in vals:
        vals.append(val)
        print(val)

17.0
20.0
22.0
24.0
29.0
30.0
32.0
34.0


In [ ]:
# Train Test Split: Train on Earlier Sessions, Test on Later Sessions

In [7]:
# Train Test Split for P8

session_threshold = 8 # train on sessions <= 8, test on sessions > 8

train = FS[(FS['session_num'] <= 8.0)] 
test = FS[(FS['session_num'] > 8)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

In [58]:
# Train Test Split for P3

session_threshold = 24 # train on sessions <= 29, test on sessions > 29

train = FS[(FS['session_num'] <= 24.0)] 
test = FS[(FS['session_num'] > 24)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

In [31]:
# Train Test Split for P7

session_threshold = 13 # train on sessions <= 13, test on sessions > 13

train = FS[(FS['session_num'] <= 13.0)] 
test = FS[(FS['session_num'] > 13)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

In [32]:
# Train Test Split for P5

session_threshold = 13 # train on sessions <= 13, test on sessions > 13

train = FS[(FS['session_num'] <= 13.0)] 
test = FS[(FS['session_num'] > 13)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

In [40]:
# Train Test Split for P9

session_threshold = 9 # train on sessions <= 9, test on sessions > 9

train = FS[(FS['session_num'] <= 9.0)] 
test = FS[(FS['session_num'] > 9)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

In [49]:
# Train Test Split for P10

session_threshold = 15.2 # train on sessions <= 15.2, test on sessions > 15.2

train = FS[(FS['session_num'] <= 15.2)] 
test = FS[(FS['session_num'] > 15.2)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

In [ ]:
X_test2.shape

### Step 2: Implement the HMM

In [8]:
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
# 3 questions from HMM:
# (1) given model, what is likelihood of sequence S happening?
# (2) given model and sequence S, what is optimal hidden state sequence?
# (3) given sequence S and # of hidden states, what is optimal model to maximize probability of S?

### P8 FS4

In [9]:
# model takes array of dimension (n_samples, n_features)
model_p8 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi').fit(X_train2)

In [10]:
#evaluate FS4 P8 HMM
FS4_pred_p8 = model_p8.predict(X_test2)
scores2 = model_p8.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P8 HMM:",accuracy_score(y_test2, FS4_pred_p8))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p8))
print(metrics.classification_report(y_test2, FS4_pred_p8))

Accuracy for FS4 P8 HMM: 0.422249076999
AUC: 0.461325220717
[[22973 41604]
 [19113 21402]]
             precision    recall  f1-score   support

        0.0       0.55      0.36      0.43     64577
        1.0       0.34      0.53      0.41     40515

avg / total       0.47      0.42      0.42    105092



### P3 FS4

In [17]:
X_train2.shape

(86393, 29)

In [59]:
# model takes array of dimension (n_samples, n_features)
model_p3 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi').fit(X_train2)

In [60]:
#evaluate FS4 P3 HMM
FS4_pred_p3 = model_p3.predict(X_test2)
scores2 = model_p3.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P3 HMM:",accuracy_score(y_test2, FS4_pred_p3))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p3))
print(metrics.classification_report(y_test2, FS4_pred_p3))

Accuracy for FS4 P3 HMM: 0.720796013624
AUC: 0.717232566481
[[ 8521  3455]
 [14251 37189]]
             precision    recall  f1-score   support

        0.0       0.37      0.71      0.49     11976
        1.0       0.91      0.72      0.81     51440

avg / total       0.81      0.72      0.75     63416



### P7 FS4

In [25]:
# model takes array of dimension (n_samples, n_features)
model_p7 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi').fit(X_train2)

#evaluate FS4 P3 HMM
FS4_pred_p7 = model_p7.predict(X_test2)
scores2 = model_p7.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P7 HMM:",accuracy_score(y_test2, FS4_pred_p7))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p7))
print(metrics.classification_report(y_test2, FS4_pred_p7))

Accuracy for FS4 P7 HMM: 0.555490146307
AUC: 0.676412808938
[[82361  4083]
 [78222 20493]]
             precision    recall  f1-score   support

        0.0       0.51      0.95      0.67     86444
        1.0       0.83      0.21      0.33     98715

avg / total       0.68      0.56      0.49    185159



### P5 FS4 HMM

In [35]:
# model takes array of dimension (n_samples, n_features)
model_p5 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi').fit(X_train2)

#evaluate FS4 P5 HMM
FS4_pred_p5 = model_p5.predict(X_test2)
scores2 = model_p5.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P5 HMM:",accuracy_score(y_test2, FS4_pred_p5))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p5))
print(metrics.classification_report(y_test2, FS4_pred_p5))

Accuracy for FS4 P5 HMM: 0.299397867608
AUC: 0.304566302833
[[ 2840  6248]
 [30287 12773]]
             precision    recall  f1-score   support

          0       0.09      0.31      0.13      9088
          1       0.67      0.30      0.41     43060

avg / total       0.57      0.30      0.36     52148



### P9 FS4 HMM

In [43]:
# model takes array of dimension (n_samples, n_features)
model_p9 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi').fit(X_train2)

#evaluate FS4 P3 HMM
FS4_pred_p9 = model_p9.predict(X_test2)
scores2 = model_p9.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P9 HMM:",accuracy_score(y_test2, FS4_pred_p9))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p9))
print(metrics.classification_report(y_test2, FS4_pred_p9))

Accuracy for FS4 P9 HMM: 0.525811753015
AUC: 0.497699591787
[[70999 36241]
 [50608 25305]]
             precision    recall  f1-score   support

          0       0.58      0.66      0.62    107240
          1       0.41      0.33      0.37     75913

avg / total       0.51      0.53      0.52    183153



### P10 FS4 HMM

In [51]:
# model takes array of dimension (n_samples, n_features)
model_p10 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi').fit(X_train2)

#evaluate FS4 P3 HMM
FS4_pred_p10 = model_p10.predict(X_test2)
scores2 = model_p10.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P10 HMM:",accuracy_score(y_test2, FS4_pred_p10))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p10))
print(metrics.classification_report(y_test2, FS4_pred_p10))

Accuracy for FS4 P10 HMM: 0.677399475438
AUC: 0.643241958526
[[38091 35070]
 [30365 99310]]
             precision    recall  f1-score   support

        0.0       0.56      0.52      0.54     73161
        1.0       0.74      0.77      0.75    129675

avg / total       0.67      0.68      0.67    202836

